# CO2-Ausstoss von Personenwagen
In diesem Notebook werden Informationen des Bundesamts für Strassen aufbereitet. Der Datensatz ist sehr umfangreich. Damit trotzdem effizient damit gearbeitet werden kann, werden bloss die für unsere Fragestellungen relevanten Daten eingelesen beziehungsweise behalten. Das genaue Vorgehen kann anhand des untenstehenden Python-Codes nachvollzogen werden. Die Rohdaten sind allerdings nicht verfügbar, da diese (kostenpflichtig) beim Bundesamt für Strassen bezogen werden müssen. 

Testdaten: https://files.admin.ch/astra_ffr/mofis/Datenlieferungs-Kunden/opendata/1000-Fahrzeuge_IVZ/1400-Vertragspflichtige_Datensaetze/1430-Bestaende_mit_PLZ/BEST_R-Testdatensatz.txt

In [49]:
# Importieren der nötigen Bibliotheken. 
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
tqdm.pandas()
from matplotlib import pyplot as plt
%matplotlib-inline

/home/mark/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
UsageError: Line magic function `%matplotlib-inline` not found.


In [2]:
# Die für unsere Fragestellung relevanten Spalten werden ausgewählt.
columns = ['Fahrzeugart', 'Leistung', 'Treibstoff', 'Erstinverkehrsetzung_Jahr', 'Altersklasse_Halter', 
           'Ort', 'BFS-Gemeinde-Nr', 'Halterart', 'CO2', 'Staat', 'Inverkehrsetzung_Kanton',
           'Marke_und_Typ']

In [3]:
# Einlesen der Daten. 
df = pd.read_csv('Rohdaten/1959_Fahrzeugbestand 2021 und 2011/BEST_R-20210501.txt', usecols=columns,  sep='\t', engine='c')

/home/mark/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (28,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Uns interessieren einzig die Personenwagen. Alle anderen Daten werden ausgefiltert.
df = df[df['Fahrzeugart'] == 'Personenwagen'].copy()

In [5]:
# Im Datensatz sind Autos enthalten, die für den Export bestimmt sind oder nicht auf die Strassen sollen. 
# Diese sind für uns irrelevant. Wir behalten einzig Fahrzeuge mit einem Schweizer Nummernschild.
df = df[df['Staat'] == 'Schweiz'].copy()

In [6]:
#Wie viele Autos ingesamt?
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4670515 entries, 20 to 6811875
Data columns (total 12 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Fahrzeugart                object 
 1   Marke_und_Typ              object 
 2   Leistung                   object 
 3   Treibstoff                 object 
 4   CO2                        float64
 5   Erstinverkehrsetzung_Jahr  int64  
 6   Inverkehrsetzung_Kanton    object 
 7   Ort                        object 
 8   BFS-Gemeinde-Nr            object 
 9   Staat                      object 
 10  Halterart                  object 
 11  Altersklasse_Halter        object 
dtypes: float64(1), int64(1), object(10)
memory usage: 463.2+ MB


In [7]:
# Bei wie vielen Personenwagen fehlt eine Angabe zum CO2-Wert? 
len(df[df['CO2'].isna()])

384746

In [8]:
# Bei wie vielen Prozent aller Autos fehlt eine Angabe zum CO2-Wert?
100 * len(df[df['CO2'].isna()]) / len(df)

8.237763929673708

In [9]:
#So viele Prozent der Autos werden im Verlauf dieses Notebooks entfernt
weg = 1986 + 326681 + 1236 + 15 + 4
100 * weg / len(df)

7.063931921854442

In [10]:
#Bei wie vielen Autos hat es weder bei Treibstoff noch bei CO2 eine Angabe?
len(df[(df['Treibstoff'].isna()) & (df['CO2'].isna())])

1986

In [11]:
#Aus welchem Jahr sind sie im Median?
df[(df['Treibstoff'].isna()) & (df['CO2'].isna())]['Erstinverkehrsetzung_Jahr'].median()

1967.0

In [12]:
# Autos, bei denen weder beim Treibstoff noch beim CO2 ein Wert erfasst sind, werden entfernt. 
df = df.dropna(subset=['Treibstoff', 'CO2'], how='all')

In [13]:
#In Welchen Gemeinden fehlt bei mehr als 20 Prozent der Autos der CO2-Wert?
#bfsliste = df['BFS-Gemeinde-Nr'].unique().tolist()

#df_control = pd.DataFrame()

#for i in tqdm(bfsliste):
#    value = 100 * len(df[(df['BFS-Gemeinde-Nr'] == i) & (df['CO2'].isna())]) / len(df[df['BFS-Gemeinde-Nr'] == i])
#    if value > 20:
#        temp_dict = {'BFS-Nummer': i, 'Autos ohne CO2_pct': [value]}
#        df_temp = pd.DataFrame(data=temp_dict)
#        df_control = pd.concat([df_control, df_temp])

#Export in ein csv
#df_control.to_csv('kontrolle_gemeinden_fehlende_co2_angaben.csv')      

In [14]:
#Bei wie vielen Autos hat es bei Verbrennungsmotoren keinen CO2-Wert?
#Autos in 'übrige Treibstoffe' haben keinen CO2-Wert. Sie werden ebenfalls entfernt, weil die Treibstoffart unklar ist
df['Treibstoff'] = df['Treibstoff'].str.strip().copy()
treibstoff = ['Benzin', 'Diesel', 'Methanol', 'Flüssiggas (LPG) / Benzin',
           'Benzin / Elektrisch', 'Diesel / Elektrisch', 'Benzin / Alkohol (Ethanol)', 'Erdgas (CNG) / Benzin', 
           'Gas (CNG/GPL)', 'Flüssiggas (LPG)', 'Erdgas (CNG)', 
           'Alkohol (Ethanol)', 'Petrol', 'übrige Treibstoffe']
len(df[((df['Treibstoff'].isin(treibstoff)) & (df['CO2'].isna()))])

326681

In [15]:
#Was ist das Median-Jahr für diese Autos?
df[((df['Treibstoff'].isin(treibstoff)) & (df['CO2'].isna()))]['Erstinverkehrsetzung_Jahr'].median()

2007.0

In [16]:
# Verbrennungsmotoren, bei denen kein CO2-Wert vorhanden ist, werden entfernt. 
df = df[~((df['Treibstoff'].isin(treibstoff)) & (df['CO2'].isna()))]

In [17]:
#Welche Treibstoffarten verbleiben im Dataset ohne CO2-Angabe?
df[df['CO2'].isna()]['Treibstoff'].unique()

array(['Elektrisch', 'Elektrisch mit RE (Range Extender)',
       'Wasserstoff / Elektrisch'], dtype=object)

In [18]:
# Bei den übrigen Autos -- alle mit mind. teilweise Elektroantrieb -- wird der fehlende CO2-Wert auf 0 gesetzt. 
# Damit werden diese Fahrzeuge in der Auswertung miteinbezogen. 
for i, r in df[df['CO2'].isna()].iterrows():
    df.at[i, 'CO2'] = 0

In [19]:
# In den Daten gibts diverse Einträge zu emmissionslosen Verbrennungsmotoren. Wie viele sind es? 
len(df[((df['CO2'] == 0) & ((df['Treibstoff'] == 'Benzin') | (df['Treibstoff'] == 'Diesel')))])

1236

In [20]:
# Verbrennungsmotoren, die kein CO2 verursachen, gibts leider nicht. Die entsprechenden Einträge werden entfernt. 
df = df[~((df['CO2'] == 0) & (df['Treibstoff'] == 'Benzin'))]
df = df[~((df['CO2'] == 0) & (df['Treibstoff'] == 'Diesel'))]

In [21]:
#Es gibt 15 Elektroautos mit einem CO2-Verbrauch über 1 g/km. Dabei dürfte es sich gemäss Astra um Fehler handeln.
df[(df['Treibstoff'] == 'Elektrisch') & (df['CO2'] > 1)].describe()

,CO2,Erstinverkehrsetzung_Jahr
count,15.000000,15.000000
mean,141.400000,2017.200000
std,184.378803,3.783422
min,12.000000,2008.000000
25%,20.000000,2016.000000
50%,121.000000,2019.000000
75%,145.500000,2020.000000
max,739.000000,2020.000000


In [22]:
#Die E-Autos mit CO2 > 1 werden aus dem Datensatz entfernt
df = df[~((df['Treibstoff'] == 'Elektrisch') & (df['CO2'] > 1))]

In [23]:
#Maximaler CO2-Ausstoss in den Daten
df.max()

Fahrzeugart                  Personenwagen
Marke_und_Typ                   vwMultivan
CO2                                    999
Erstinverkehrsetzung_Jahr             2021
Inverkehrsetzung_Kanton                 ZH
Ort                                locarno
BFS-Gemeinde-Nr                        996
Staat                              Schweiz
Halterart                         weiblich
dtype: object

In [24]:
#Es gibt vier Autos mit einem unglaubwürdig hohen CO2-Ausstoss von 999 g/km. Gemäss Astra ist die 999
#ausserdem die höchstmögliche Zahl in ihrem System. Es dürfte sich also um einen Fehler handeln
df[df['CO2'] == 999]

,Fahrzeugart,Marke_und_Typ,Leistung,Treibstoff,CO2,Erstinverkehrsetzung_Jahr,Inverkehrsetzung_Kanton,Ort,BFS-Gemeinde-Nr,Staat,Halterart,Altersklasse_Halter
34175,Personenwagen,ALFA ROMEO GiuliettaSprint13,59,Benzin,999.0,1965,BL,Gelterkinden,2'846,Schweiz,männlich,70 bis 79
1061204,Personenwagen,CITROEN C5 2.2 HDI,150,Diesel,999.0,2012,ZH,Mettmenstetten,9,Schweiz,männlich,50 bis 59
1069541,Personenwagen,CITROEN DS3 1.6 HDI,84,Diesel,999.0,2014,FR,Châtel-St-Denis,2'325,Schweiz,weiblich,20 bis 29
5650043,Personenwagen,Toyota Auris,73,Benzin / Elektrisch,999.0,2019,ZH,Zürich,261,Schweiz,männlich,60 bis 69


In [25]:
#Die Wagen mit CO2 == 999 g/km werden entfernt
df = df[~(df['CO2'] == 999)]

In [26]:
# Im Datensatz sind Gemeinden enthalten, bei denen die nicht existierende Gemeindenummer 0 eingetragen wurde. 
# Wir versuchen anhand der übrigen Daten die richtigen Nummern einzutragen. 
df['BFS-Gemeinde-Nr'] = df['BFS-Gemeinde-Nr'].apply(lambda x: int(str(x).replace('\'', '')))

for i, r in tqdm(df[df['BFS-Gemeinde-Nr'] == 0].iterrows()):
    try:
        b = df[df['Ort'] == r['Ort']]['BFS-Gemeinde-Nr'].value_counts().index.tolist()[0]
        df.at[i, 'BFS-Gemeinde-Nr'] = int(b)
    except: 
        pass

# Weitere Korrektur: Bei allen Einträgen zu Ponte Cremenaga, einem Weiler in der Gemeinde Monteggio, 
# fehlen die BfS-Nummern. 
for i, r in tqdm(df[df['Ort'].str.lower() == 'ponte cremenaga'].iterrows()):
    df.at[i, 'BFS-Gemeinde-Nr'] = 5202

### Test: Gemeinden mit mehreren BFS-Nummern

In [27]:
# Welchen Gemeinden sind mehrere BfS-Nummern zugeordnet? 
df_test = pd.DataFrame(df.groupby(by=['Ort', 'Inverkehrsetzung_Kanton'])['BFS-Gemeinde-Nr'].nunique())
df_test = df_test[df_test['BFS-Gemeinde-Nr'] != 1]
df_test = df_test.reset_index()

In [28]:
# Eine Spalte für die korrigierte BFS-Nummer wird eingefügt. 
for i, r in tqdm(df_test.iterrows()):
    bfs = df[(df['Ort'] == r['Ort']) & (df['Inverkehrsetzung_Kanton'] == r['Inverkehrsetzung_Kanton'])]['BFS-Gemeinde-Nr'].value_counts().index.tolist()[0]
    df_test.at[i, 'bfs_korr'] = bfs
    
df_test.sort_values(by='BFS-Gemeinde-Nr', ascending=False)

,Ort,Inverkehrsetzung_Kanton,BFS-Gemeinde-Nr,bfs_korr
91,Lausanne,VD,14,5586.0
119,Morges,VD,9,5642.0
15,Bern,BE,8,351.0
69,Gland,VD,7,5721.0
33,Cheseaux-sur-Lausanne,VD,6,5582.0
...,...,...,...,...
76,Hertenstein,AG,2,4038.0
77,Hofstetten,BE,2,580.0
78,Innertkirchen,BE,2,784.0
79,Jens,BE,2,738.0


In [29]:
df[df['Ort'] == 'Lausanne']['BFS-Gemeinde-Nr'].unique()

array([5586, 5590, 5635, 5633, 5484, 5410, 5889, 5648, 5587, 5721, 5803,
       5613, 5591, 5588])

Im Datensatz existieren Gemeinden, denen mehr als eine BFS-Nummer zugeordnet wurde. Die Abklärung beim Astra ergab, dass die Gemeinden anhand der BFS-Nummer bearbeitet werden sollen. Sie ist die verlässliche Angabe im Gegensatz zur Spalte "Ort".

### Daten-Export

In [30]:
# Exportieren der gefilterten Komplettdaten. 
df.to_csv('co2_daten_gefiltert.csv')

In [31]:
# Exportieren der gefilterten Daten gruppiert nach Gemeinde.
df_export = df.groupby(by='BFS-Gemeinde-Nr').agg({'Ort': 'first', 'CO2': 'mean', 'Treibstoff': 'count'})
df_export.columns = ['ort', 'CO2', 'anz_fahrzeuge']
df_export = df_export[df_export.index > 0].copy()

Obacht: beim Groupby wird für den Ort immer der erste Wert genommen. Zum Beispiel Aarau Rohr für die bfs-Nummer 4001, die zu Aarau gehört. Diese Unschärfe spielt insofern keine Rolle, als im nächsten Notebook 2_raumgliederung der aktuelle Gemeindestand hinzugemergt wird. In der zusätzlichen, neuen Ortsspalte steht dann Aarau.

In [32]:
df_export.to_csv('co2_daten_gefiltert_gruppiert.csv')